In [1]:
import os
import sys
_path = os.path.abspath('../')
if _path not in sys.path:
    sys.path.append(_path)

import astropy.coordinates as coord
import astropy.table as at
from astropy.io import fits
import astropy.units as u
import gala.coordinates as gc

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import healpy as hp
from tqdm import tqdm, trange
func = lambda x, y, z: hp.vec2pix(nside, x, y, z)

from helpers import projs, get_data

In [2]:
def get_vscale(img, q=[5, 95]):
    from astropy.stats import sigma_clip
    
    _tmp = img.ravel()
    stdfunc = lambda x,axis: 1.5*np.median(np.abs(x-np.median(x,axis=axis)), axis=axis)
    _tmp_clipped = sigma_clip(_tmp[_tmp != 0], stdfunc=stdfunc)
    
    return np.percentile(_tmp_clipped[_tmp_clipped > 0], q)

def zyx_euler_from_endpoints(lon1, lat1, lon2, lat2):
    c1 = coord.SkyCoord(lon1*u.deg, lat1*u.deg)
    c2 = coord.SkyCoord(lon2*u.deg, lat2*u.deg)
    fr = gc.GreatCircleICRSFrame.from_endpoints(c1, c2)
    origin = fr.realize_frame(coord.UnitSphericalRepresentation(0*u.deg, 0*u.deg))

    gc_icrs = origin.transform_to(coord.ICRS)
    R = gc.greatcircle.reference_to_greatcircle(coord.ICRS, fr)
    psi = -np.degrees(np.arctan2(R[2,1], R[2,2]))
    
    return [gc_icrs.ra.degree, gc_icrs.dec.degree, psi]

In [3]:
cube, dms, footprint_mask = get_data(bass_file='../data/BASS_iso_hpxcube_z0.0001_a13.5_gmax23.fits.gz', 
                                     decals_file='../data/DECaLS_iso_hpxcube_z0.0001_a13.5_gmax23.fits.gz')
npix, nslice = cube.shape
nside = hp.npix2nside(npix)

In [4]:
proj = hp.projector.GnomonicProj(xsize=1024, ysize=512, 
                                 rot=[23.55, -30.52, -43.3],
                                 reso=3.)

In [10]:
import os

def make_anim_frames(cube, proj, name, mu_lim=None,
                     vscale=None, sigma=None, figsize=None,
                     save_path=None):
    """
    Parameters
    ----------
    cube : numpy.ndarray
        The full cube of filtered healpix maps.
    proj : healpy projection
        The healpy projection instance.
    name : str
        Stream name
    mu_lim : tuple (optional)
        The distance modulus limits to animate over. If not 
        specified, this defaults to the full range in the cube.
    vscale : dict, tuple (optional)
        If a dictionary, this will be passed to get_vscale() as 
        keyword arguments. If a tuple, this is assumed to be a
        tuple of (vmin, vmax) values to hard-set for all frames.
    sigma : float [radians] (optional)
        Smoothing.
    figsize : tuple (optional)
        If not set, this is determined automatically with the 
        xsize and ysize of the projection.
    save_path : str (optional)
        
    
    """

    if sigma is None:
        sigma = np.radians(0.12)  # MAGIC NUMBER
    
    if mu_lim is None:
        mu_lim = (dms.min(), dms.max())
    
    mu_idx = (np.argmin(np.abs(dms - mu_lim[0])),
              np.argmin(np.abs(dms - mu_lim[1])))
    
    if figsize is None:
        proj_info = proj.get_proj_plane_info()
        xsize = proj_info.get('xsize', 1.)
        ysize = proj_info.get('ysize', xsize)
        figsize = (10, 10 * ysize / xsize)
    
    if save_path is None:
        save_path = '.'
    
    # Hacks...
    if isinstance(vscale, dict) or vscale is None:
        vscale_func = get_vscale
        
        if vscale is None:
            vscale_kw = {}
        else:
            vscale_kw = vscale
    
    else:
        vscale_func = lambda *args, **_: vscale
        vscale_kw = {}
        
    for j, i in enumerate(np.arange(mu_idx[0], mu_idx[1]+1, 1)):
        print(f'm-M = {dms[i]:.1f}')
    
        hpxmap = cube[:, i]
        hpxmap_smooth = hp.smoothing(hpxmap, sigma=sigma, verbose=False)

        img = proj.projmap(hpxmap_smooth, func)

        vmin, vmax = vscale_func(img, **vscale_kw)
        print(f'vmin, vmax = ({vmin:.2f}, {vmax:.2f})')

        fig, ax = plt.subplots(1,1, figsize=(12, 6))
        ax.imshow(img, origin='bottom', vmin=vmin, vmax=vmax, cmap='Greys', 
                  extent=proj.get_extent())
        ax.set_title(f'm-M = {dms[i]:.1f}')
        fig.savefig(os.path.join(save_path, f'{name}_{j:02d}.png'), dpi=250)
        plt.close(fig)

In [15]:
make_anim_frames(cube, proj, name='atlas', mu_lim=(15, 18))

m-M = 15.0
vmin, vmax = (0.00, 4.76)
m-M = 15.1
vmin, vmax = (0.00, 4.92)
m-M = 15.2
vmin, vmax = (0.00, 5.13)
m-M = 15.3
vmin, vmax = (0.00, 5.39)
m-M = 15.4
vmin, vmax = (0.00, 5.70)
m-M = 15.5
vmin, vmax = (0.00, 6.08)
m-M = 15.6
vmin, vmax = (0.00, 6.54)
m-M = 15.7
vmin, vmax = (0.00, 7.06)
m-M = 15.8
vmin, vmax = (0.00, 7.60)
m-M = 15.9
vmin, vmax = (0.00, 8.10)
m-M = 16.0
vmin, vmax = (0.00, 8.57)
m-M = 16.1
vmin, vmax = (0.00, 8.94)
m-M = 16.2
vmin, vmax = (0.00, 9.20)
m-M = 16.3
vmin, vmax = (0.00, 9.33)
m-M = 16.4
vmin, vmax = (0.00, 9.34)
m-M = 16.5
vmin, vmax = (0.00, 9.23)
m-M = 16.6
vmin, vmax = (0.00, 9.01)
m-M = 16.7
vmin, vmax = (0.00, 8.70)
m-M = 16.8
vmin, vmax = (0.00, 8.35)
m-M = 16.9
vmin, vmax = (0.00, 7.94)
m-M = 17.0
vmin, vmax = (0.00, 7.51)
m-M = 17.1
vmin, vmax = (0.00, 7.06)
m-M = 17.2
vmin, vmax = (0.00, 6.59)
m-M = 17.3
vmin, vmax = (0.00, 6.11)
m-M = 17.4
vmin, vmax = (0.00, 5.61)
m-M = 17.5
vmin, vmax = (0.00, 5.13)
m-M = 17.6
vmin, vmax = (0.00, 4.66)
m